In [30]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import itertools
from nltk.tokenize import RegexpTokenizer
from functools import partial

In [21]:
# conv.txt
tokenizer = RegexpTokenizer(r'\w+')
def process_sentence(x, eos=False):
#     x = x.split()
    x = tokenizer.tokenize(x)
    x = [i.lower() for i in x]
    if eos:
        x = ['eos'] + x + ['eos']
    return x

def one_hot(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
    
def sent_encode(x):
    x = [w_i[i] for i in x]
    x = map(one_hot, x)
    return list(x)

data=pd.read_csv('conv.txt', delimiter='\t', error_bad_lines=False, header=None)
data.dropna(inplace=True)
inp = list(map(partial(process_sentence), data[0].values))
out = list(map(partial(process_sentence, eos=True), data[1].values))
words=np.r_[inp, out]
words = list(set(itertools.chain(*words)))
vocab_size=len(words)
i_w = dict(enumerate(words))
w_i = {v:k for k, v in i_w.items()}
inp = np.array(list(map(sent_encode, inp)))
out = np.array(list(map(sent_encode, out)))
print(vocab_size, inp.shape, out.shape)

b'Skipping line 1681: expected 2 fields, saw 3\nSkipping line 1765: expected 2 fields, saw 3\nSkipping line 1984: expected 2 fields, saw 3\nSkipping line 2046: expected 2 fields, saw 3\nSkipping line 2086: expected 2 fields, saw 3\nSkipping line 2313: expected 2 fields, saw 3\nSkipping line 2314: expected 2 fields, saw 3\n'


3325 (2330,) (2330,)


In [25]:
batch_size = 20
input_size = len(inp[0][0])
dim = 256
input_size, dim
print_step = 20
iterations = 1000

In [23]:
max_length1 = max([len(x) for x in inp])
max_length2 = max([len(x) for x in out])
print(max_length1, max_length2)
def get_batch(size):
    idx = np.random.choice(range(len(inp)), size=size, replace=False)
    x, y, y_ = inp[idx], list(map(lambda x: x[:-1], out[idx])), list(map(lambda x: x[1:], out[idx]))
    x = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length1 - len(z))], x))
    y = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length2 - len(z))], y))
    y_ = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length2 - len(z))], y_))
    x, y, y_ = np.array(x), np.array(y), np.array(y_)
    return x, y, y_
a, b, c = get_batch(5)
# print(np.argmax(b[0], axis=1))
# print(np.argmax(c[0], axis=1))

70 139


In [32]:
tf.reset_default_graph()
with tf.Graph().as_default() as graph:
    def length(sequence):
        ''' returns length of sequences in a batch '''
        used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length
    
    def cost(output, target):
        cross_entropy = target * tf.log(output)
        cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
        mask = tf.sign(tf.reduce_max(tf.abs(target), reduction_indices=2))
        cross_entropy *= mask
        cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.reduce_sum(mask, reduction_indices=1)
        return tf.reduce_mean(cross_entropy)
    
    def compute_loss(y, t, l):
        batch_loss = [tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y[i], t[i]))
                      / tf.cast(l[i], tf.float32)
                      for i in range(batch_size)]
        batch_loss_mean = tf.reduce_mean(batch_loss)
        return batch_loss_mean
    
    def trim_zeros(seq, length):
        x_ = [tf.slice(seq[i], [0, 0], [length[i], input_size]) for i in range(batch_size)]
        return x_
    
    def display_seq(seq):
        def seq_id_to_word(x):
            return [i_w[i] for i in x]
        temp=list(map(np.trim_zeros, np.argmax(seq, axis=2)))
        temp = list(map(seq_id_to_word, temp))
        for t in temp:
            print(' '.join(t))
    
    with tf.variable_scope('encoder'):
        enc_seq = tf.placeholder(tf.float32, [batch_size, max_length1, input_size])
        enc_seq_length = length(enc_seq)
        enc_out, enc_final = tf.nn.dynamic_rnn(tf.nn.rnn_cell.GRUCell(dim), enc_seq, 
                                      sequence_length=enc_seq_length, dtype=tf.float32)
        
    with tf.variable_scope('decoder'):    
        dec_tar = tf.placeholder(shape=[batch_size, max_length2, input_size], dtype=tf.float32)
        dec_seq = tf.placeholder(tf.float32, [batch_size, max_length2, input_size])
        dec_seq_length = length(dec_seq)
        dec_out = tf.nn.dynamic_rnn(tf.nn.rnn_cell.GRUCell(dim), 
                                        dec_seq, sequence_length=dec_seq_length, initial_state=enc_final)[0]
        
    Wy = tf.Variable(tf.random_uniform(shape=[dim, input_size],
                                      minval=-0.1, maxval=0.1), dtype=tf.float32)
    by = tf.Variable(tf.zeros(shape=[input_size]))
    yp = tf.einsum('ijk,kl->ijl', dec_out, Wy) + by
    yp_t = trim_zeros(yp, dec_seq_length)
    tar_t = trim_zeros(dec_tar, dec_seq_length)
    loss = compute_loss(yp_t, tar_t, dec_seq_length)
    optimize_op = tf.train.AdagradOptimizer(0.01).minimize(loss)
    saver = tf.train.Saver()
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True), graph=graph) as sess:
#         sess.run(tf.initialize_all_variables())
        new_saver = tf.train.import_meta_graph('model.ckpt.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./'))
        avg_loss = 0
        
        for i in range(iterations):
            a, b, c = get_batch(batch_size)
            _, l = sess.run([optimize_op, loss], {enc_seq: a, dec_seq: b, dec_tar: c})
            avg_loss += l
            if i % print_step == 0 and i != 0:
                print(avg_loss / print_step)                
                o, t = sess.run([yp_t, tar_t], {enc_seq: a, dec_seq: b, dec_tar: c})
                for i in o[:5]:
                    print(np.argmax(i, axis=1))
                avg_loss = 0
                saver.save(sess, 'model.ckpt')

5.84926898479
[ 767 1524 1524 1524 1524 1524 1524]
[ 767 1524 1524 1524 1524 1524 1524]
[ 767 1226]
[ 767 1524 1524 1524 1524]
[ 767 1524 1524 1524 1524 1524 1524 1524 1524 1524]
5.43368396759
[ 767 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524
 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1226 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524
 1524]
[ 767 1226 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1226]
5.57905776501
[ 767 1226]
[ 767 1524 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1226 1524 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1226 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524 1524]
[ 767 1226 1524 1524 1524 1524 1524 1524 1524 1524]


KeyboardInterrupt: 